In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input,Conv2D, LeakyReLU, UpSampling2D, Conv2DTranspose, Add, ReLU
from keras import backend as K

In [ ]:
filters = 64
D = 5
R = 8
kernel = (3,3)
def build_conv(D,filters,kernel):
    _in = Input((None,None,64))
    x = Conv2D(filters, kernel,
               padding='same',
               kernel_initializer = 'he_normal',
               name='conv1',
               use_bias=False)(_in)
    x = LeakyReLU(alpha=0.2)(x)
    for idx in range(D-1):
        x = Conv2D(filters, kernel,
                   padding='same',
                   kernel_initializer = 'he_normal',
                   name='conv'+str(idx+2),
                   use_bias=False)(x)
        x = LeakyReLU(alpha=0.2)(x)
    return Model(_in,x)
conv = build_conv(D,filters,kernel)

In [ ]:
def feature_embedding(shared_conv):
    _in = Input((None,None,64))
    # Conv1
    x = shared_conv(_in)
    return Model(_in,x)

In [ ]:
def bilinear_init(shape):
    # https://github.com/tensorlayer/tensorlayer/issues/53
    num_channels = shape[3]
    bilinear_kernel = np.zeros([shape[0], shape[1]], dtype=np.float32)
    scale_factor = (shape[0] + 1) // 2
    if shape[1] % 2 == 1:
        center = scale_factor - 1
    else:
        center = scale_factor - 0.5
    for x in range(shape[0]):
        for y in range(shape[1]):
            bilinear_kernel[x,y] = (1 - abs(x - center) / scale_factor) * (1 - abs(y - center) / scale_factor)
    weights = np.zeros((shape[0], shape[1], num_channels, num_channels))
    for i in range(num_channels):
        weights[:, :, i, i] = bilinear_kernel
    return K.variable(weights)

def feature_upsampling(): 
    _in = Input((None,None,64))
    x = Conv2DTranspose(64,(4,4),
                        strides=2,
                        padding='same',
                        kernel_initializer = bilinear_init,
                        use_bias=False,
                        name='feat_up')(_in)
    return Model(_in,x)

def image_upsampling():
    _in = Input((None,None,3))
    x = Conv2DTranspose(3,(4,4),
                        strides=2,
                        padding='same',
                        kernel_initializer = bilinear_init,
                        use_bias=False,
                        name='img_up')(_in)
    return Model(_in,x)

In [ ]:
def conv_res(R,shared_conv):
    
    _in = Input((None,None,64))
    
    x = shared_conv(_in)
    for idx in range(R-1):
        x = shared_conv(x)
    
    x = Conv2D(3,(3,3),
               strides=1,
               padding='same',
               activation='sigmoid',
               kernel_initializer = 'he_normal',
               name='sub_band1',
               use_bias=False)(x)
    
    return Model(_in,x)


In [ ]:
shared_feat_emb = feature_embedding(conv)
shared_feat_up = feature_upsampling()
shared_conv_res = conv_res(R,conv)
shared_img_up = image_upsampling()

In [ ]:
shared_feat_emb.summary()

In [ ]:
184+65+65

In [ ]:
def build_LapSRN():
    shared_conv = build_conv(D,filters,kernel)
    shared_feat_emb = feature_embedding(shared_conv)
    shared_feat_up = feature_upsampling()
    shared_conv_res = conv_res(8,shared_conv)
    shared_img_up = image_upsampling()
    _in = Input((64,64,3))
    x = Conv2D(64,(3,3),strides=1,padding='same',name='conv_in',kernel_initializer = 'he_normal')(_in)
    x = LeakyReLU(alpha=0.2)(x)
    
    feat_emb_2x = shared_feat_emb(x)
    feat_up_2x = shared_feat_up(feat_emb_2x)
    sub_band_res_2x = shared_conv_res(feat_up_2x)
    img_up_2x = shared_img_up(_in)
    hr_2x_out = Add()([sub_band_res_2x,img_up_2x])
    
    
    feat_emb_4x = shared_feat_emb(feat_up_2x)
    feat_up_4x = shared_feat_up(feat_emb_4x)
    sub_band_res_4x = shared_conv_res(feat_up_4x)
    img_up_4x = shared_img_up(hr_2x_out)
    hr_4x_out = Add()([sub_band_res_4x,img_up_4x])
    
    return Model(_in,[hr_2x_out,hr_4x_out])

In [ ]:
def loss_function(y_true, y_pred):
    '''
    y_true -> Is the array of bicubic downsampled HR images at each level
    y_pred -> Is the addition of sub band residuals from conv_res and upsampling ie img_up
    '''
    r = y_true - y_pred
    # Charbonnier penalty
    epsilon = 1e-3
    p = K.sqrt(K.square(r) + K.square(epsilon))
    return K.mean(p)

In [ ]:
LapSRN = build_LapSRN()

In [ ]:
LapSRN.summary()

In [ ]:
253-222

In [ ]:
LapSRN.compile(loss=loss_function, optimizer='sgd')

# Testing

In [1]:
envPath = '../'
import sys
sys.path.append(envPath+'model/')


In [2]:
from LapSRN import LapSRN as network

Using TensorFlow backend.


In [3]:
LapSRN = network(optimizer='sgd')

In [4]:
LapSRN_model = LapSRN.get_LapSRN()

W1204 21:36:58.077900 139780360537920 deprecation_wrapper.py:119] From /home/pratik/anaconda3/envs/srPath/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1204 21:36:58.080211 139780360537920 deprecation_wrapper.py:119] From /home/pratik/anaconda3/envs/srPath/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1204 21:36:58.088945 139780360537920 deprecation_wrapper.py:119] From /home/pratik/anaconda3/envs/srPath/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1204 21:36:58.142784 139780360537920 deprecation.py:323] From /home/pratik/anaconda3/envs/srPath/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:2618: Variable.initialized_value (from t

In [5]:
LapSRN_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv_in (Conv2D)                (None, 64, 64, 64)   1792        input_6[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_6 (LeakyReLU)       (None, 64, 64, 64)   0           conv_in[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 multiple             184640      leaky_re_lu_6[0][0]              
                                                                 model_3[1][0]                    
__________